# Prediction Algorithms | Verb Prediction

***Please find the assignments to submit following the sample code.***

This notebook contains working code as the basis for your assignment. It fetches and loads the necessary data you will work on during this assignment, and demonstrates all the initial steps.

## Assignment Overview

In this assignment, you will learn to use classification algorithms ― Naive Bayes and Logistic Regression ― to deduce whether a word is a verb or not. You will leverage these algorithms to classify yet-unseen words into verbs and non-verbs, and work to improve their rate of success. 

To do so, you will practice the art of feature engineering ― selecting and implementing features that give the algorithm sufficient power to gain good performance. You will practice the use of evaluation metrics, and work iteratively towards arriving at good results.

## Technical Prerequisites
+ **Python 3.6.x.** You may install python via the [Anaconda distribution](https://www.anaconda.com/distribution/#download-section) which will also automatically install many useful python libraries, but this is not mandatory. To import a library in a notebook or python file, it must first be _installed_; this is a prerequisite to running this notebook. Anaconda will out-of-the-box install most/all of the libraries which are imported by this notebook. The latest version of Anaconda comes with python 3.7, so you will have to dig for the previous version of Anaconda.
<br><Br>
+ Python packages used in this notebook should be installed. <br><br>
    + To install any missing or helpful python packages with Anaconda, you would typically issue the following command given below. This command assumes that the package you are after has been made publically available on the official Anaconda distribution channel (called conda-forge). The `conda` command is Anaconda's setup tool that comes along with the Anaconda distribution. If you installed Anaconda, you should have this command available in your command prompt.
```
conda install -c conda-forge <package name>
```
    + Further setup notes:
         + When run, the `conda install` command may take a while to resolve all dependencies.
         + Some packages are not available through Anaconda, in which case use the famous pip utility to install the missing package/s into your python environment.
         + You are entirely free to install python and the necessary packages in other ways
<br><br>

## About Universal Dependencies Treebanks

In this assignment you will use treebanks as the input data. Let us discuss what is a treebank.

+ A treebank is a linguistic type of corpus that captures the structure of sentences.
+ It also captures the category that each word belongs to ― its Part of Speech.
+ Typically, a treebank has been created through labour-intensive annotation.

Specifically, in this assignment we will use treebanks which follow the modern [Universal Dependencies](https://universaldependencies.org/introduction.html) standard!

You will learn much more about these concepts in later stages of this course. For now, we will only use them in a somewhat simplistic way.

## Getting Started with this notebook
+ Follow the cells and their outputs to confirm you understand what they are doing. 
+ Use it as a learning opportunity about idiomatic python, if you are relatively new to python!
+ Make a local copy of the notebook ― <br>and make sure it is running on your machine, with outpus similar to those already appearing in this pre-run copy.

### Imports

In [1]:
import pyconll  # library parsing Universal Dependencies files given in CoNLL-U format
import random
import itertools 
import sklearn.metrics
from sklearn.metrics import confusion_matrix
from tqdm import tqdm_notebook # progress bar
from pprint import pprint  # slightly nicer printing of data structures

## Downloading and loading the HTB corpus
To future-proof, we standardize on a specific version of the HTB. <br>Those with a keen eye will notice that in the following we use the quick-and-dirty ⚡ way of running OS commands from directly within the notebook. <br>⚙ Anyway, make sure you have git installed and working on your OS before proceeding.

In [77]:
%%script false 
!git clone https://github.com/UniversalDependencies/UD_Hebrew-HTB
!cd UD_Hebrew-HTB && git checkout 82591c955e86222e32531336ff23e36c220b5846

In [78]:
conllu1 = pyconll.load_from_file('UD_Hebrew-HTB/he_htb-ud-dev.conllu')
conllu2 = pyconll.load_from_file('UD_Hebrew-HTB/he_htb-ud-test.conllu')
conllu3 = pyconll.load_from_file('UD_Hebrew-HTB/he_htb-ud-train.conllu')

conllu = [conllu1, conllu2, conllu3]

## Quick data exploration
Let us quantify some basic properties of the data, such as how many verbs do we have per sentence, to get a basic grasp of the data, before diving into our tasks. This is typically called the Data Exploration phase.

### Verbs v.s. Non Verbs

#### counting the number of verbs per sentence, and making a histogram of that

In [4]:
counts = []
for sentence in itertools.chain(*conllu): # the asterisk unpacks the array into a function argument list
    verbs = 0
    for token in sentence:
        if token.upos == 'VERB':
            verbs += 1 
    
    counts.append(verbs)

#### a histogram view of the counts
here we exemplify use of the [pandas library](https://towardsdatascience.com/pandas-series-a-lightweight-intro-b7963a0d62a2). pandas is _the_ python library for series and tabular data exploration and manipulation. 

In [5]:
import pandas as pd 

# turning the array we got, into a pandas series
counts = pd.Series(counts)

# getting the number of sentences that have n verbs for every n
hist = counts.value_counts().sort_index().to_frame('sentences count')
hist.index.name = '# of verbs'
hist

,sentences count
# of verbs,
0,511
1,1673
2,1748
3,1099
4,604
5,321
6,140
7,66
8,26


#### split the words into verbs and non-verbs

In [7]:
verbs = {}
non_verbs = {}

for sentence in itertools.chain(*conllu):
    for token in sentence:
        if token.upos == 'VERB':
            if token.form in verbs:
                verbs.update({token.form : verbs[token.form]+1})
            else:
                verbs.update({token.form : 0})
        else:
            if token.form in non_verbs:
                non_verbs.update({token.form : non_verbs[token.form]+1})
            else:
                non_verbs.update({token.form : 0})


# the next line uses the python set union operator
ambiguous = set(verbs.keys()) & set(non_verbs.keys()) 
                
# the (right-hand side of the) next couple of lines are dict comprehensions ― 
# eseentially that's just a one liner syntax for a loop
verbs     = {k:v for (k,v) in verbs.items() if not k in ambiguous} 
non_verbs = {k:v for (k,v) in non_verbs.items() if not k in ambiguous}

In [8]:
print('there are {:,} unique verbs in the data'.format(len(verbs)))
print('there are {:,} unique non-verbs in the data'.format(len(non_verbs)))
print('{:,} words appear in the data as both a verb and a non-verb'.format(len(ambiguous)))

there are 4,896 unique verbs in the data
there are 27,573 unique non-verbs in the data
632 words appear in the data as both a verb and a non-verb


In [9]:
print('proportion of verbs in the overall lexicon: {:.3f}'.format(len(verbs) / (len(verbs) + len(non_verbs) + len(ambiguous))))

proportion of verbs in the overall lexicon: 0.148


### Character Ngrams

#### extracting all ngrams (up to length 2) from all the words

In [34]:
ngrams = dict()
max_ngram_len = 2

lexicon = list(verbs.keys()) + list(non_verbs.keys())

# for each verb (tqdm is just a nice utility for getting a progress bar for a long loop)
for verb in tqdm_notebook(lexicon):  
    # per ngram length
    for ngram_len in range(1,max_ngram_len): 
        # sliding window iterate its ngrams
        for idx in range(len(verb) - ngram_len + 1):
            ngram = verb[idx : idx + 2] 

            if ngram in ngrams:
                ngrams[ngram] += 1
            else:
                ngrams[ngram] = 1        

In [36]:
print("the ngrams:\n")
print(str(sorted(ngrams.keys())))

the ngrams:

['!', '"', '"א', '"ב', '"ג', '"ד', '"ה', '"ו', '"ז', '"ח', '"ט', '"י', '"ך', '"כ', '"ל', '"ם', '"מ', '"ן', '"נ', '"ס', '"ע', '"ף', '"פ', '"ץ', '"צ', '"ק', '"ר', '"ש', '"ת', '%', '(', ')', ',', ',0', ',1', ',2', ',3', ',4', ',5', ',6', ',7', ',8', ',9', '-', '.', '..', '.0', '.1', '.2', '.3', '.4', '.5', '.6', '.7', '.8', '.9', '.א', '.ב', '.ד', '.כ', '.ס', '.פ', '.ר', '0', '0,', '0.', '00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '1', '1,', '1.', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '1פ', '2', '2,', '2.', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '2ח', '2ש', '3', '3,', '3.', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '4,', '4.', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '5,', '5.', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '6,', '6.', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '6א', '7', '7,', '7.', '70', '71', '72', '73', '74', '75', 

#### basic counts

In [35]:
print('{} ngrams unique extracted from the data'.format(len(ngrams)))
_1grams = list(filter(lambda ngram: len(ngram) == 1, ngrams))
_2grams = list(filter(lambda ngram: len(ngram) == 2, ngrams))
print('{} 2-grams'.format(len(_2grams)))
print('{} 1-grams'.format(len(_1grams)))

921 ngrams unique extracted from the data
873 2-grams
48 1-grams


#### ngram frequencies

In [43]:
hist_data = pd.DataFrame.from_dict(ngrams, orient='index', columns=['occurences'])

print("ngram frequencies:")
with pd.option_context("display.max_rows", 10):
    display(hist_data)

ngram frequencies:


,occurences
מג,181
גי,572
יע,513
עי,704
ים,3677
...,...
2ש,1
ש2,1
2ח,1
נ.,2


#### frequencies histogram
trivially showing that ngrams that occur more often are fewer

In [55]:
# some imports and setup for plotting
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from colab import enable_plotly_in_colab

init_notebook_mode(connected=True)
enable_plotly_in_colab()

data = [go.Histogram(x=hist_data['occurences'])]
fig = go.Figure(data=data, layout=go.Layout(
        yaxis=dict(title='ngrams with that frequency'), 
        xaxis=dict(title='ngram frequency')))

iplot(fig)

## The Training Pipeline
In the following we test whether ngrams are enough for classifying words into verbs v.s. non-verbs. This demonstrates the entire flow of classification.

### Vectorization
Vectorization is the turning of inputs into feature vectors; in our case, turning words into feature vectors. 

In [56]:
def vectorize(word):
    ''' turn the given word into a feature vector '''
    
    # ngram occurences
    vec1 = [0] * len(ngrams)
    for idx, ngram in enumerate(ngrams):
        if ngram in word:
            if vec1[idx]:
                vec1[idx] += 1
            else:
                vec1[idx] = 1
            
    # ngram occurences as prefix
    vec2 = [0] * len(ngrams)
    for idx, ngram in enumerate(ngrams):
        if word.startswith(ngram):
            if vec2[idx]:
                vec2[idx] += 1
            else:
                vec2[idx] = 1


    # ngram occurences as suffix
    vec3 = [0] * len(ngrams)
    for idx, ngram in enumerate(ngrams):
        if word.endswith(ngram):
            if vec3[idx]:
                vec3[idx] += 1
            else:
                vec3[idx] = 1
        
    return vec1 + vec2 + vec3

A spec of our feature vector; This is a map mapping each index of our feature vectors to the meaning of the feature stored on that index. It must be kept aligned to the vectorize function above, and one could make a class encompassing both, under the object oriented programming paradigm.

In [71]:
feature_vector_description = \
    [(ngram, 'occurs') for ngram in enumerate(ngrams)] + \
    [(ngram, 'startswith') for ngram in enumerate(ngrams)] + \
    [(ngram, 'endswith') for ngram in enumerate(ngrams)]

def feature_to_description(idx):
    return feature_vector_description[idx]

### Train Test Split
We split the the data into a train and a test split

In [57]:
def train_test_split(X, y, train_proportion=0.8):
    ''' splits the given data into train and test sets '''

    assert len(X) == len(y), 'input data should have exactly one prediction per input'
    assert 0 < train_proportion < 1, 'this function requires a proportion between zero and one as its second argument'
    
    data_indices = set(range(len(X)))
    data_count = len(data_indices)

    train_indices = set(random.sample(data_indices, int(data_count * train_proportion)))
    test_indices  = data_indices - train_indices
    
    X_train = [X[idx] for idx in train_indices]
    X_test  = [X[idx] for idx in test_indices]
    
    y_train = [y[idx] for idx in train_indices]
    y_test  = [y[idx] for idx in test_indices]

    assert len(X_train) + len(X_test) == len(X)
    
    return X_train, X_test, y_train, y_test

### Arranging the data for classifier training input 

+ We whip our vectorized data into a matrix $X$ of feature vectors. Each row of the matrix then represents an input sample ― in our case a word ― through its feature vector that we have computed. 

+ We also derive a _vector_ $y$ of the correct class value per input sample; in our case, $0$ representing a non-verb and $1$ representing a verb).

Order matters! the semantics are that the $nth$ row of $X$ has the class label given as the $nth$ element of $y$.

In [61]:
X_positive = list(map(vectorize, tqdm_notebook(verbs.keys())))
X_negative = list(map(vectorize, tqdm_notebook(non_verbs.keys())))

y_positive = [1] * len(X_positive)
y_negative = [0] * len(X_negative)

X_train_pos, X_test_pos, y_train_pos, y_test_pos = train_test_split(X_positive, y_positive)
X_train_neg, X_test_neg, y_train_neg, y_test_neg = train_test_split(X_negative, y_negative)

X_train = X_train_pos + X_train_neg
y_train = y_train_pos + y_train_neg

Maybe we want to bias the classifier with sample weights, but for now this code applies uniform sample weights

In [65]:
pos_sample_weight = 1
neg_sample_weight = 1
sample_weights = ([pos_sample_weight] * len(X_train_pos)) + ([neg_sample_weight] * len(X_train_neg))

### Fitting the model
**This is the model training step (a.k.a the model fitting)**.<br>
We use the classification algorithm implementations of [scikit learn](https://scikit-learn.org/stable/), python's default machine learning library. 
<br>With Naive Bayes, and roughly in general, the more features and data we have, the longer this will take to complete.

In [66]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

model = MultinomialNB()
#model = LogisticRegression()

model.fit(X_train, y_train, sample_weights)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

#### peeking into the feature importances of the fitted model

In [72]:
model_coefficients = list(zip(model.coef_[0], feature_vector_description))
sorted(list(model_coefficients), reverse=True, key=lambda tuple: tuple[0])[:12]

[(-3.282431357960295, ((118, 'ו'), 'occurs')),
 (-3.351714042519845, ((239, 'י'), 'occurs')),
 (-3.6525254064134183, ((17, 'ה'), 'occurs')),
 (-3.6710513160209324, ((97, 'ת'), 'occurs')),
 (-3.783333428757265, ((531, 'מ'), 'occurs')),
 (-3.80704326083632, ((53, 'ל'), 'occurs')),
 (-3.908887164021568, ((39, 'ר'), 'occurs')),
 (-4.060195587079896, ((734, 'נ'), 'occurs')),
 (-4.182126285256361, ((531, 'מ'), 'startswith')),
 (-4.217663019533148, ((57, 'ש'), 'occurs')),
 (-4.265291068522403, ((118, 'ו'), 'endswith')),
 (-4.360465756151194, ((71, 'ע'), 'occurs'))]

### So we have a trained (fitted) model now. How good is it?
looking at specific examples can be tempting, but it doesn't scale as well as making a full evaluation

In [74]:
words = ['הלבשה', 'התלבשו', 'התלבשה', 'הלבישה', 'התלבש', 'לבש', 'לבוש']
list(zip(words, (model.predict(list(map(vectorize, words))))))

[('הלבשה', 0),
 ('התלבשו', 1),
 ('התלבשה', 1),
 ('הלבישה', 0),
 ('התלבש', 1),
 ('לבש', 1),
 ('לבוש', 0)]

### To really evaluate our model, we will let it predict the labels of our test set 

In [ ]:
X_test = X_test_pos + X_test_neg
y_test = y_test_pos + y_test_neg

y_pred = model.predict(X_test)

## Model Evaluation

In [ ]:
sklearn.metrics.accuracy_score(y_test, y_pred)

In [ ]:
print(sklearn.metrics.classification_report(y_test, y_pred))

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
sklearn.metrics.precision_score(y_test, y_pred, average='micro')

# Assignment

## _3 pts_ | Question 1 
To understand how well (or badly) a model is performing and to gauge progress in improving it, it is good to come up with a baseline. Unless there is already a previously known or public benchmark result to compare to, a naive benchmark is a good idea for getting started. Assume you have no benchmark to compare to, and you hence wish to have a baseline. ***suggest and calculate a reasonable even if naive, baseline over the test set***. hint: think of the class distribution in the data.*

## _2 pts_ | Question 2 
what can you say about the performance obtained here, according to your baseline?

## _3 pts_ | Question 3
in light of the above, suggest and implement (from scratch) an adjusted accuracy metric, that would better account for the class imbalance inherent in the data. class imbalance is defined as a case where the distribution of classes in the data is far from being uniform.

<span style="color:red">Note: you cannot use any ready-made evaluation metric from sklearn or any other library for this question</span>

## _8 pts_ | Question 4
suggest and implement better features based on properties of the Hebrew language. implement the features and provide a notebook with your implementation and the resulting performance report showing the performance improvement.

<span style="color:red">Note that you are not allowed to use any of the annotations of the original corpus in this exercise.</span>

## _4 pts_ | Question 5
now alternatively suggest and apply features specifically for English. Provide a notebook with your implementation classifying over the English data of https://github.com/UniversalDependencies/UD_English-EWT, including a performance report.

*use the following code cell to download and use the chosen version of this data*

In [ ]:
%%script false
!git clone https://github.com/UniversalDependencies/UD_English-EWT
!cd UD_English-EWT && git checkout 7be629932192bf1ceb35081fb29b8ecb0bd6d767

## _30 pts_ | Question 6 
_In this question you will implement a classifier that takes account of the context of a word_. This means, you will no longer classify a word, context-less, but rather, you will transition to classifying words as part of the sentence they appear in. It will open the way for bringing in additional features into play in your model, and very hopefully, lead to substantially improved classification performance.

+ You may reuse the code of this notebook in any way, or write from scratch.

+ You will be graded based upon:

   + dilligence and creativity in adding reasonable features
   + score on the test set, obtained when run for grading
   + reproducibility: does your submitted solution notebook actually run when downloaded for grading
   + cleanly structured, mildly self-explanatory or documented code
   
<span style="color:red">Note that you are not allowed to use any of the annotations of the original corpus in this exercise.</span>

## _20 pts_ | Question 7
Implement the _Multinomial Naive Bayes_ algorithm itself from scratch. Establish the same performances as obtained with the ready-made Multinomial Naive Bayes algorithm that you have used so far.

What to submit for this?
+ a working notebook using your own implemented Naive Bayes classifier rather than the sklearn one, per each of the programming assignments you have already implemented above


## _20 pts_ | Question 8
Implement the _logistic regression_ algorithm itself from scratch. Establish the same performances as obtained with the ready-made logistic regression algorithm that you have used so far.

What to submit for this?
+ a working notebook using your own implemented logistic regression algorithm rather than the sklearn one, per each of the programming assignments you have already implemented above


## _10 pts_ | Question 9
*Manual domain adaptation.* 
<br>
1. obtain an unrelated Hebrew dataset; 

    + this can be a liberated copy of your gmail inbox ([google data takeout](https://takeout.google.com/settings/takeout))
    + a whatsapp data export
    + public hebrew tweets
    + the hebrew wikipedia


2. Extract 50 sentences of that data
<br><br>
3. Use your best accomplished model over that subset of the data.

<br><br>
What to submit for this?
1. The 50 sentences and your annotation of them (only annotate verb / non-verb per word)
2. Your evaluation over them (also 'comparing to the original results)
3. Now try to analyze toward a qualitative description of the difference in the performance; Submit your analysis.


+ _15 pts bonus:_ modify the training data to improve performance over your data. submit your modified training data here as well.

<span style="color:gray">what not to include in your submission?</span>
+ <span style="color:gray">You may not include in your new data, any data explicitly or implicitly disclosing the identity or personal information of any person/s. Anonymize any non-public data that you use, as necessary to comply with this requirement.</span>
+ <span style="color:gray">You may not include any proprietary or confidential data in your new data.</span>

## <span style="color:orange">Submission Notes</span>

+ <span style="color:red">Note that non-reproducible results will be ignored.</span>
+ <span style="color:red">Any submission that does not actually run (halts with an error) will immediately deduce 5 pts. Any re-submission that does not actually run (halts with an error) will deduce additional 5 pts each. So make sure to verify that your code submission runs before submitting it.</span>
+ <span style="color:gray">if any of your code should take more than 10 minutes to run, require more than 16GB of memory, or happen to rely on a GPU, please provide a notifcation in advance.

The way to avoid this troubles, is to bundle your solution into a compressed zip archive, unzip it to an empty folder, and verify it is running from scratch from there.

